In [34]:
#Importing the required libraries
import pandas as pd
import numpy as np
import pyvista as pv
from pyvista import CellType

##### There are 234 points in the dataset and for each we have the data of: Barhole Number, Barhole ID, Longitude and Latitude, depth, O2 Concentration and the measured above sea level.
##### The points are supposed to be visualized with their amount of O2 concentration while the above sea level changes.

In [37]:
df = pd.read_csv('Desktop/WorkDocs/AGW/data_Marzieh/Charmant/O2_data/Excel/NewDataset.csv')

In [38]:
#Drop the columns we don't need them for creating a mesh(GPS coordinates are prefered)
df = df.rename(columns={'OxCons': 'O2Con', 'OxPer':'AboSeaLev'})
df_v1 = df.drop(['UTMLong', 'UTMLat'], axis=1)

In [39]:
df_v1

,BarhNo,BarhID,GPSLong,GPSLat,Depth,O2Con,AboSeaLev
0,HW22,113.00,8.412087,49.048153,8.53,7.088,104.47
1,HW22,113.00,8.412087,49.048153,9.53,5.409,103.47
2,HW22,113.00,8.412087,49.048153,10.53,4.701,102.47
3,HW22,113.00,8.412087,49.048153,11.53,4.765,101.47
4,HW22,113.00,8.412087,49.048153,12.53,4.672,100.47
...,...,...,...,...,...,...,...
229,T125,115.74,8.376957,49.005508,7.55,3.709,108.19
230,T125,115.74,8.376957,49.005508,8.55,2.940,107.19
231,T125,115.74,8.376957,49.005508,9.55,2.137,106.19
232,T125,115.74,8.376957,49.005508,10.55,1.647,105.19


In [42]:
#Adding a column to keep the rounded above sea level values
rounded_AboSeaLev = df_v1.AboSeaLev.round()
df_v1.insert(7, "r_AboSeaLev", rounded_AboSeaLev, True)

In [43]:
df_v1

,BarhNo,BarhID,GPSLong,GPSLat,Depth,O2Con,AboSeaLev,r_AboSeaLev
0,HW22,113.00,8.412087,49.048153,8.53,7.088,104.47,104.0
1,HW22,113.00,8.412087,49.048153,9.53,5.409,103.47,103.0
2,HW22,113.00,8.412087,49.048153,10.53,4.701,102.47,102.0
3,HW22,113.00,8.412087,49.048153,11.53,4.765,101.47,101.0
4,HW22,113.00,8.412087,49.048153,12.53,4.672,100.47,100.0
...,...,...,...,...,...,...,...,...
229,T125,115.74,8.376957,49.005508,7.55,3.709,108.19,108.0
230,T125,115.74,8.376957,49.005508,8.55,2.940,107.19,107.0
231,T125,115.74,8.376957,49.005508,9.55,2.137,106.19,106.0
232,T125,115.74,8.376957,49.005508,10.55,1.647,105.19,105.0


##### Lets Call rounded values of above sea level, r_AboSeaLev from now on!
##### In the following block the number of unique r_AboSeaLev is counted and we will sea the number of r_AboSeaLev reported for each Barhole.

In [47]:
unique_values = df_v1['r_AboSeaLev'].unique()  # Get unique values
value_counts = df_v1['r_AboSeaLev'].value_counts()  # Count the occurrences of each unique value

# Print unique values and their counts
print("Unique values:", unique_values)
print("\nr_AboSeaLev Value counts:")
print(value_counts)

Unique values: [104. 103. 102. 101. 100.  99.  98.  97.  96. 105. 111. 110. 109. 108.
 107. 106. 113. 112.  95.  94.  93.  92.  91.  90.  89.  88.  87.  86.
  85.  84.  83.  82.  81.  80.  79.  78.  76.]

r_AboSeaLev Value counts:
104.0    23
105.0    19
103.0    19
106.0    17
107.0    16
108.0    16
102.0    16
109.0    14
101.0    14
100.0    11
98.0      9
99.0      9
110.0     9
97.0      8
111.0     6
96.0      5
112.0     2
113.0     2
86.0      1
78.0      1
79.0      1
80.0      1
81.0      1
82.0      1
83.0      1
84.0      1
85.0      1
95.0      1
87.0      1
88.0      1
89.0      1
90.0      1
91.0      1
92.0      1
93.0      1
94.0      1
76.0      1
Name: r_AboSeaLev, dtype: int64


##### To create a precise visualization we have to fill the non-reported values out. Some AboSeaLevs are reported in more than 10 Barholes and that make sence to create visualisations for them, then at least we will have the interpolation in 66% of the data and less. So the r_AboSeaLevs that has reported values in more than 10 Barholes are kept.

In [6]:
filtered_df = df_v1[df_v1['r_AboSeaLev'].isin([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110])]

In [9]:
filtered_df

,BarhNo,BarhID,GPSLong,GPSLat,Depth,O2Con,AboSeaLev,r_AboSeaLev
0,HW22,113.00,8.412087,49.048153,8.53,7.088,104.47,104.0
1,HW22,113.00,8.412087,49.048153,9.53,5.409,103.47,103.0
2,HW22,113.00,8.412087,49.048153,10.53,4.701,102.47,102.0
3,HW22,113.00,8.412087,49.048153,11.53,4.765,101.47,101.0
4,HW22,113.00,8.412087,49.048153,12.53,4.672,100.47,100.0
...,...,...,...,...,...,...,...,...
229,T125,115.74,8.376957,49.005508,7.55,3.709,108.19,108.0
230,T125,115.74,8.376957,49.005508,8.55,2.940,107.19,107.0
231,T125,115.74,8.376957,49.005508,9.55,2.137,106.19,106.0
232,T125,115.74,8.376957,49.005508,10.55,1.647,105.19,105.0


In [48]:
# Group the data by 'BarhNo' and count the unique 'r_AboSeaLev' for each group
unique_values_per_barhno = filtered_df.groupby('BarhNo')['r_AboSeaLev'].nunique()

# Check if all 'BarhNo' have 10 unique 'r_AboSeaLev' values
unique_values_per_barhno_full_coverage = unique_values_per_barhno == 11
all_have_ten = unique_values_per_barhno_full_coverage.all()

In [49]:
# List of all possible unique values for 'r_AboSeaLev'
all_possible_values = list(range(int(filtered_df['r_AboSeaLev'].min()), int(filtered_df['r_AboSeaLev'].max()) + 1))

# Finding missing 'r_AboSeaLev' values for each 'BarhNo'
missing_values_per_barhno = {
    barhno: [value for value in all_possible_values if value not in group['r_AboSeaLev'].unique()]
    for barhno, group in filtered_df.groupby('BarhNo')
}

missing_values_per_barhno

{'HW22': [105, 106, 107, 108, 109, 110],
 'HW34': [106, 107, 108, 109, 110],
 'HW40': [105, 106, 107, 108, 109, 110],
 'HW45': [105, 106, 107, 108, 109, 110],
 'HW49': [105, 106, 107, 108, 109, 110],
 'HW55': [105, 106, 107, 108, 109, 110],
 'HW60': [105, 106, 107, 108, 109, 110],
 'T101': [],
 'T102': [100, 101, 102, 103, 104],
 'T104': [],
 'T105': [100, 101, 102, 103, 104, 105],
 'T106': [100, 101, 102, 103, 104, 105],
 'T107': [100, 101, 102, 103, 109, 110],
 'T109': [100, 101, 109, 110],
 'T111': [100, 101, 102, 107, 108, 109, 110],
 'T112': [100, 101, 106, 107, 108, 109, 110],
 'T113': [100, 101, 102, 103, 108, 109, 110],
 'T115': [100, 107, 108, 109, 110],
 'T117': [100, 101, 107, 108, 109, 110],
 'T118': [100, 106, 107, 108, 109, 110],
 'T123': [100, 101, 102, 110],
 'T124': [100, 101, 102, 110],
 'T125': [100, 101, 102, 103, 104, 110],
 'T128': [100, 101, 102, 103, 104, 105, 106],
 'T129': [100, 101, 102, 103, 104, 105, 106, 107, 108],
 'T320': [105, 106, 107, 108, 109, 110],


In [50]:
# Create a template row based on the existing data structure with zero values
template_row = {column: 0 for column in filtered_df.columns}

# Initialize an empty list to collect new rows
new_rows = []

# Generate missing rows for each 'BarhNo'
for barhno, missing_values in missing_values_per_barhno.items():
    for value in missing_values:
        new_row = template_row.copy()
        new_row['BarhNo'] = barhno
        new_row['r_AboSeaLev'] = value
        # Handling the other required fields
        previous_row = filtered_df[filtered_df['BarhNo'] == barhno].iloc[0]  # Assuming there's only one row per 'BarhNo'
        new_row['GPSLong'] = previous_row['GPSLong']
        new_row['GPSLat'] = previous_row['GPSLat']
        new_rows.append(new_row)

# Convert list of new rows to DataFrame
new_rows_df = pd.DataFrame(new_rows)

# Preview the new rows to be added
new_rows_df.head()

,BarhNo,BarhID,GPSLong,GPSLat,Depth,O2Con,AboSeaLev,r_AboSeaLev
0,HW22,0,8.412087,49.048153,0,0,0,105
1,HW22,0,8.412087,49.048153,0,0,0,106
2,HW22,0,8.412087,49.048153,0,0,0,107
3,HW22,0,8.412087,49.048153,0,0,0,108
4,HW22,0,8.412087,49.048153,0,0,0,109


In [51]:
# Function to find the closest existing `r_AboSeaLev` value and return its `O2Con`
def find_closest_O2Con(barhno, target_r_abosealev):
    # Filter the original data for the given BarhNo
    subset = filtered_df[filtered_df['BarhNo'] == barhno]
    # Find the `r_AboSeaLev` value closest to the target
    closest_r_abosealev = subset.iloc[(subset['r_AboSeaLev'] - target_r_abosealev).abs().argsort()[:1]]
    return closest_r_abosealev['O2Con'].values[0]

# Update `O2Con` in new rows using the closest `r_AboSeaLev`'s `O2Con`
for new_row in new_rows:
    new_row['O2Con'] = find_closest_O2Con(new_row['BarhNo'], new_row['r_AboSeaLev'])

# Convert list of new rows to DataFrame again with updated `O2Con`
new_rows_df_updated = pd.DataFrame(new_rows)

# Preview the updated new rows DataFrame
new_rows_df_updated.head()

,BarhNo,BarhID,GPSLong,GPSLat,Depth,O2Con,AboSeaLev,r_AboSeaLev
0,HW22,0,8.412087,49.048153,0,7.088,0,105
1,HW22,0,8.412087,49.048153,0,7.088,0,106
2,HW22,0,8.412087,49.048153,0,7.088,0,107
3,HW22,0,8.412087,49.048153,0,7.088,0,108
4,HW22,0,8.412087,49.048153,0,7.088,0,109


In [12]:
# Append the new rows to the original dataset
combined_data = pd.concat([filtered_df, new_rows_df_updated], ignore_index=True)

# Verify the combination by checking unique 'r_AboSeaLev' values count per 'BarhNo' again
verified_unique_values_per_barhno = combined_data.groupby('BarhNo')['r_AboSeaLev'].nunique()

# Preview the verification result and the combined dataframe shape
verified_unique_values_per_barhno, combined_data.shape

(BarhNo
 HW22    11
 HW34    11
 HW40    11
 HW45    11
 HW49    11
 HW55    11
 HW60    11
 T101    11
 T102    11
 T104    11
 T105    11
 T106    11
 T107    11
 T109    11
 T111    11
 T112    11
 T113    11
 T115    11
 T117    11
 T118    11
 T123    11
 T124    11
 T125    11
 T128    11
 T129    11
 T320    11
 T411    11
 T412    11
 T517    11
 T524    11
 Name: r_AboSeaLev, dtype: int64,
 (342, 8))

In [52]:
#For some barholes in some abovSeaLev there are 2 or more O2 concentrations reported, so we need to drop the duplicated values and keep the first one.
drDop_data = combined_data.drop_duplicates(subset=['BarhNo','r_AboSeaLev'] , keep='first')
drDop_data = drDop_data.reset_index(drop= True)

In [53]:
drDop_data

,BarhNo,BarhID,GPSLong,GPSLat,Depth,O2Con,AboSeaLev,r_AboSeaLev
0,HW22,113.0,8.412087,49.048153,8.53,7.088,104.47,104.0
1,HW22,113.0,8.412087,49.048153,9.53,5.409,103.47,103.0
2,HW22,113.0,8.412087,49.048153,10.53,4.701,102.47,102.0
3,HW22,113.0,8.412087,49.048153,11.53,4.765,101.47,101.0
4,HW22,113.0,8.412087,49.048153,12.53,4.672,100.47,100.0
...,...,...,...,...,...,...,...,...
325,T524,0.0,8.454053,48.999730,0.00,0.508,0.00,101.0
326,T524,0.0,8.454053,48.999730,0.00,0.508,0.00,102.0
327,T524,0.0,8.454053,48.999730,0.00,0.508,0.00,103.0
328,T524,0.0,8.454053,48.999730,0.00,0.508,0.00,104.0


In [16]:
drDop_data.groupby('BarhNo')['r_AboSeaLev'].count()

BarhNo
HW22    11
HW34    11
HW40    11
HW45    11
HW49    11
HW55    11
HW60    11
T101    11
T102    11
T104    11
T105    11
T106    11
T107    11
T109    11
T111    11
T112    11
T113    11
T115    11
T117    11
T118    11
T123    11
T124    11
T125    11
T128    11
T129    11
T320    11
T411    11
T412    11
T517    11
T524    11
Name: r_AboSeaLev, dtype: int64

In [18]:
df_sorted = drDop_data.sort_values(by=['BarhNo', 'r_AboSeaLev'])

In [19]:
df_sorted = df_sorted.sort_values(by=['BarhNo', 'r_AboSeaLev']).reset_index(drop=True)

In [68]:
df_sorted.to_csv('finaldataset.csv' , index=False)

In [54]:
Long = np.array(df_sorted.GPSLong)
Lat = np.array(df_sorted.GPSLat)

#Normalizing the values of Longitude and Latitude in df to a range between (0,1)
def normalize(arr, t_min, t_max):
    norm_arr = []
    diff = t_max - t_min
    diff_arr = max(arr) - min(arr)
    for i in arr:
        temp = (((i - min(arr))*diff)/diff_arr) + t_min
        norm_arr.append(temp)
    return norm_arr
 
#assign array and range for Longitude
array_1d = Long
range_to_normalize = (0, 1)

normalized_Long = normalize(
    array_1d, range_to_normalize[0], 
  range_to_normalize[1])
 
#assign array and range for latitude 
array_1d = Lat
range_to_normalize = (0, 1)

normalized_Lat = normalize(
    array_1d, range_to_normalize[0], 
  range_to_normalize[1])

#Creating points using the normalized Coordinates
Points = np.stack((normalized_Long, normalized_Lat), axis=1)

#Getting indexes from our points to create the mesh
indices_Upper_depth = df_sorted.index[df_sorted['r_AboSeaLev'] == 100.0].tolist()
mask = np.isin(np.arange(len(Points)), indices_Upper_depth)

# Use the boolean mask to filter the points
selected_points = Points[mask]

#Creating a triangulated mesh using Delaunay package
from scipy.spatial import Delaunay

tri = Delaunay(selected_points)
triangle_points_list = []

for simplex in tri.simplices:
    #triangle_points = points[simplex]
    triangle_points_list.append(simplex)

triangle_points_array = np.array(triangle_points_list)

In [55]:
#In this block we need to read the BarhNo assigned to each point and save them in an array for labling the points in the visualization blocks
# Select rows from the DataFrame corresponding to the indexes in your list
selected_rows = df_sorted.loc[indices_Upper_depth]

# Extract the values from the 'barhNo' column of the selected rows
barhNo_values = selected_rows['BarhNo'].values

In [56]:
def OxygenConcentration(df,points,triangulated_points,AboSeaLev,barhNo_values):
    
    lower_triangulated_points = triangulated_points + 30
    wedge_points = np.concatenate((lower_triangulated_points, triangulated_points), axis=1)
    
    # To define the cells of a mesh we need to keep the number of the points in a cell in the first column of the triangle_points_array.
    new_column = np.full((wedge_points.shape[0], 1), 6)

    # Concatenate the new column with the original array to define the Mesh Cells
    Cells_array = np.concatenate((new_column, wedge_points), axis=1)
    
    #To define a 3rd dimention for points in the upper face of the wedge
    Converted_Upper_face_point_to3d = np.column_stack((points, np.full(points.shape[0], 0.1)))
    #To define a 3rd dimention for points in the Lower face of the wedge
    Converted_Lower_face_point_to3d = np.column_stack((points, np.full(points.shape[0], 0)))
    # Total points of the wedge
    total_wedge_points = np.vstack((Converted_Upper_face_point_to3d, Converted_Lower_face_point_to3d))
    
    # Get the Oxygen concentration value of the points regarding the mentioned depth
    Upper_face_O2Con_values = df.O2Con[df['r_AboSeaLev'] == AboSeaLev + 1].tolist()
    Lower_face_O2Con_values = df.O2Con[df['r_AboSeaLev'] == AboSeaLev].tolist()
    total_O2Con_values = np.concatenate((np.array(Upper_face_O2Con_values), np.array(Lower_face_O2Con_values)))
    
    #Define Celltype as an input argument for pyvista unstructured grid
    celltypes = np.full(50, fill_value=CellType.WEDGE, dtype=np.uint8)
    
    #Define an Unstructured grid in pyvista package
    grid = pv.UnstructuredGrid(Cells_array, celltypes, total_wedge_points)
    grid.point_data['Oxygen_Concentration(mg/L)'] = total_O2Con_values
    pl = pv.Plotter()
    pl.add_mesh(grid, show_edges=True, line_width=2)
    
    # Generate point labels based on barhNo_values
    num_upper_face_points = Converted_Upper_face_point_to3d.shape[0]
    point_labels = []

    for i in range(num_upper_face_points):
        if i < len(barhNo_values):
            point_labels.append(str(barhNo_values[i]))
        else:
            point_labels.append("")

    point_labels.extend(["" for _ in range(num_upper_face_points, len(total_wedge_points))])

    label_coords = grid.points + [0, 0, 0.02]
    #point_labels = [f'{i}' for i in range(grid.n_points)]

    
    pl.add_point_labels(label_coords, point_labels, font_size=15, point_size=10)
    pl.show()
    pl.export_html('interactive_plot.html')

In [66]:
#Here we ask the user to enter the aboveSeaLev 
valid_AbovSeaLev = {100, 101, 102, 103, 104, 105, 106, 107, 108, 109}

# Get user input for depth
while True:
    try:
        User_AboveSeaLev = int(input("Enter the Above Sea Level you are interested to see the Oxygen concetration in ,valid Above Sea Level values are (100, 101, 102, 103, 104, 105, 106, 107, 108): "))
        if User_AboveSeaLev in valid_AbovSeaLev:
            break
        else:
            print("Invalid value. Please enter a valid value.")
    except ValueError:
        print("Invalid input. Please enter a valid integer.")

# Call the function with the user-provided level
OxygenConcentration(df_sorted, selected_points, triangle_points_array, User_AboveSeaLev,barhNo_values)


Enter the Above Sea Level you are interested to see the Oxygen concetration in ,valid Above Sea Level values are (100, 101, 102, 103, 104, 105, 106, 107, 108):  109


Widget(value="<iframe src='http://localhost:50499/index.html?ui=P_0x28c52d630_16&reconnect=auto' style='width:…